In [5]:
import json
import csv
import glob
import pickle
import pandas as pd

In [6]:
scraped_data = '/Users/nyxinsane/Documents/Work - UvA/Automating Equity/Study 1/Study1_Code/scraped_data'

site_list=['Indeed', 'Glassdoor', 'LinkedIn']

cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']



In [21]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))

In [22]:
len(glob_paths)

670

In [24]:
glob_path

'/Users/nyxinsane/Documents/Work - UvA/Automating Equity/Study 1/Study1_Code/scraped_data/LinkedIn/Data/linkedin_jobs_dict_accommodation.json'

In [23]:
for glob_path in glob_paths:
    if glob_path.endswith('.json'):
        df_temp = pd.read_json(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path)

ValueError: Expected object or value

In [ ]:
for glob_path in glob_paths:
    if glob_path.endswith('.json'):
        df_temp = pd.read_json(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(columns=df_temp.filter(regex='Unnamed').columns, axis='columns', inplace=True, errors='ignore')
    
    if glob_path.endswith('.json'):
        df_temp.to_json(glob_path, orient='records', index=False)
    elif glob_path.endswith('.csv'):
        df_temp.to_csv(glob_path, orient='records', index=False)

In [ ]:
#     if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
#             df_temp = fix_keywords(df_temp)
#             df_temp.reset_index(drop=True, inplace=True)
#             if df_temp.columns.isin(cols).any() == True:
#                 df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
#                 df_temp.drop(columns=df_temp.filter(regex='Unnamed').columns, axis='columns', inplace=True, errors='ignore')
#             appended_data.append(df_temp)   